# 🚀 Fitness-AQA Vision Pipeline (YOLOv8-Pose Edition)

This version replaces MMPose with **YOLOv8**, making it much faster to install and run, while maintaining identical output for the modeling team.

## 📋 Features:
- **Instant Setup:** No complex C++ dependencies.
- **Identical Output:** 17 COCO keypoints (same as Vishal expected).
- **Smooth Signals:** Includes Savitzky-Golay smoothing.
- **Normalisation:** Includes scale-invariant coordinate scaling.

---

## 📦 Step 1: Install Ultralytics

Unlike MMPose, this only takes ~30 seconds and **doesn't** require a restart.

In [ ]:
!pip install ultralytics scipy opencv-python matplotlib -q
!pip install "numpy<2.0.0" -q

import torch
from ultralytics import YOLO

print(f"✅ YOLOv8 Installed!")
print(f"✅ GPU Available: {torch.cuda.is_available()}")

## 📤 Step 2: Upload Your Video

In [ ]:
from google.colab import files
import os

uploaded = files.upload()
if uploaded:
    video_path = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {video_path}")
else:
    print("❌ No file uploaded.")

## 🔧 Step 3: Pipeline Logic

In [ ]:
import json
import numpy as np
from scipy.signal import savgol_filter

class YOLOPoseExtractor:
    def __init__(self, model_variant='yolov8s-pose.pt', device='cuda'):
        self.model = YOLO(model_variant)
        self.device = device

    def smooth_signal(self, keypoints, window_length=5, polyorder=2):
        if len(keypoints) < window_length: return keypoints
        smoothed = np.zeros_like(keypoints)
        for i in range(keypoints.shape[1]):
            smoothed[:, i, 0] = savgol_filter(keypoints[:, i, 0], window_length, polyorder)
            smoothed[:, i, 1] = savgol_filter(keypoints[:, i, 1], window_length, polyorder)
        return smoothed

    def normalize_signal(self, keypoints):
        normalized = np.zeros_like(keypoints)
        for f in range(len(keypoints)):
            frame_kps = keypoints[f]
            mid_shoulder = (frame_kps[5] + frame_kps[6]) / 2
            mid_hip = (frame_kps[11] + frame_kps[12]) / 2
            torso_len = np.linalg.norm(mid_shoulder - mid_hip)
            scale = 1.0 if torso_len < 1e-3 else 1.0 / torso_len
            normalized[f] = (frame_kps - mid_hip) * scale
        return normalized

    def process_video(self, video_path):
        results = self.model(video_path, stream=True, device=self.device, verbose=False)
        raw_keypoints, scores = [], []
        
        for result in results:
            if result.keypoints is not None and len(result.keypoints.data) > 0:
                kp_data = result.keypoints.data[0].cpu().numpy()
                raw_keypoints.append(kp_data[:, :2])
                scores.append(result.keypoints.conf[0].cpu().numpy())
            else:
                raw_keypoints.append(np.zeros((17, 2)))
                scores.append(np.zeros(17))

        raw_keypoints = np.array(raw_keypoints)
        smoothed = self.smooth_signal(raw_keypoints)
        normalized = self.normalize_signal(smoothed)
        
        return {
            "video_id": os.path.basename(video_path),
            "frame_count": len(raw_keypoints),
            "raw_keypoints": raw_keypoints.tolist(),
            "smoothed_keypoints": smoothed.tolist(),
            "normalized_keypoints": normalized.tolist(),
            "scores": np.array(scores).tolist()
        }

print("✅ YOLO Logic Ready!")

## 🚀 Step 4: Run Extraction

In [ ]:
extractor = YOLOPoseExtractor()
result = extractor.process_video(video_path)

output_file = 'pullup_analysis.json'
with open(output_file, 'w') as f:
    json.dump(result, f)

print(f"\n✅ Done! Processed {result['frame_count']} frames.")
print(f"💾 Saved to: {output_file}")

## 💾 Step 5: Download for Vishal

In [ ]:
from google.colab import files
files.download('pullup_analysis.json')
print("✅ Download started! Send this to Vishal.")